# Proposal for New Office Location for Toronto Business Coders

We are starting a consulting business for data science services and java development services in Toronto, and looking to open an office with a start-up theme.  The target is to find the best location for the office, in order to maximize productivity return on employee selection, with the leading thought being that proximity to coffee shops impacts productivity.  I propose to find support for that idea, against apartment rental costs, and then to locate candidate neighborhoods accordingly.

The data we use to support this inquiry are: Foursquare location data for Toronto neighborhoods and coffee businesses; Toronto apartment pricing data from
https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods.

# Import libraries

In [36]:
import requests 
import pandas as pd 
import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 
import xml
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



# Toronto neighborhood demographics (wikipedia to dataframe)

In [37]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [38]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [39]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df4 = df3[df3.Borough != 'Not assigned']
df5 = df4.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True)
df6 = df5.replace("Not assigned", "Queen's Park")
df6.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


# Foursquare details

In [40]:
ClIENT_ID = 'A33R5RDKUNUOOIVEHZ1HS2NKGGJRXDOT13KCLQKHV3UFVD4K' # Foursquare ID
ClIENT_SECRET = 'PE1JSVLQQCNSL54HFFMDJWVUROHXDVY2IJHXMMAZ4JPSGILF' # Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: A33R5RDKUNUOOIVEHZ1HS2NKGGJRXDOT13KCLQKHV3UFVD4K
Foursquare_Secret:PE1JSVLQQCNSL54HFFMDJWVUROHXDVY2IJHXMMAZ4JPSGILF


In [41]:
# define the city and get its latitude & longitude 
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


# Find Coffee Shops

In [42]:
search_query = 'Coffee'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=A33R5RDKUNUOOIVEHZ1HS2NKGGJRXDOT13KCLQKHV3UFVD4K&client_secret=PE1JSVLQQCNSL54HFFMDJWVUROHXDVY2IJHXMMAZ4JPSGILF&ll=43.653963,-79.387207&v=20180604&query=Coffee&radius=500&limit=30'

In [43]:
search_query2 = 'Cafe'
radius = 500

# Define the corresponding URL
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=A33R5RDKUNUOOIVEHZ1HS2NKGGJRXDOT13KCLQKHV3UFVD4K&client_secret=PE1JSVLQQCNSL54HFFMDJWVUROHXDVY2IJHXMMAZ4JPSGILF&ll=43.653963,-79.387207&v=20180604&query=Cafe&radius=500&limit=30'

In [44]:
# Send the GET Request and examine the results
results = requests.get(url2).json()

In [20]:
# assign relevant part of JSON to coffee shops
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,4b0d2ef1f964a520664423e3,480 University Ave,CA,Toronto,Canada,at Dundas St W,179,"[480 University Ave (at Dundas St W), Toronto ...","[{'label': 'display', 'lat': 43.65511700442771...",43.655117,-79.388764,M5G 1V2,ON,Avenue Café + Bistro,v-1564848070
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,52306d48498e7f7573f576fa,225 Richmond St. W,CA,Toronto,Canada,at Duncan St.,541,"[225 Richmond St. W (at Duncan St.), Toronto O...","[{'label': 'display', 'lat': 43.64934659382636...",43.649347,-79.389338,M5V 2C5,ON,The Fifth Cafe,v-1564848070
2,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4f513029e4b07c3382c9fdb9,250 Dundas Street West,CA,Toronto,Canada,Simcoe Street,192,"[250 Dundas Street West (Simcoe Street), Toron...","[{'label': 'display', 'lat': 43.65457125894357...",43.654571,-79.389450,M5T 2Z5,ON,Cafe Plenty,v-1564848070
3,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,58e716e5fa7ed04f61194722,201 Dundas St W,CA,Toronto,Canada,NaN,102,"[201 Dundas St W, Toronto ON M5G 1C8, Canada]","[{'label': 'display', 'lat': 43.654872, 'lng':...",43.654872,-79.387429,M5G 1C8,ON,"Café De Paris, Downtown Toronto",v-1564848070
4,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4ad4c060f964a52064f720e3,317 Dundas Street W,CA,Toronto,Canada,NaN,347,"[317 Dundas Street W, Toronto ON M5T 1G4, Canada]","[{'label': 'display', 'lat': 43.65405398907735...",43.654054,-79.391525,M5T 1G4,ON,CafeAGO,v-1564848070
5,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",False,4ab989ebf964a520cc7f20e3,208 Queen St. W,CA,Toronto,Canada,NaN,426,"[208 Queen St. W, Toronto ON M5V 1Z2, Canada]","[{'label': 'display', 'lat': 43.650401, 'lng':...",43.650401,-79.389156,M5V 1Z2,ON,Queen Mother Cafe,v-1564848070
6,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4aec87eff964a52058c821e3,24 Baldwin St.,CA,Toronto,Canada,at Henry St,553,"[24 Baldwin St. (at Henry St), Toronto ON M5T ...","[{'label': 'display', 'lat': 43.65621134762937...",43.656211,-79.393331,M5T 1L2,ON,Cafe La Gaffe,v-1564848070
7,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,5ab82b0d62420b32095c8ff1,201 Dundas Street West,CA,Toronto,Canada,NaN,99,"[201 Dundas Street West, Toronto ON M5G 1C8, C...","[{'label': 'display', 'lat': 43.65485, 'lng': ...",43.654850,-79.387330,M5G 1C8,ON,One Pear Cafe,v-1564848070
8,"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",False,4e5cecc9fa76a4cf148bc565,162 McCaul St.,CA,Toronto,Canada,btwn D'Arcy & Baldwin,434,"[162 McCaul St. (btwn D'Arcy & Baldwin), Toron...","[{'label': 'display', 'lat': 43.65577205054815...",43.655772,-79.391993,M5T 1W3,ON,Krispy Kreme Doughnut Cafe,v-1564848070
9,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",False,5241f2d911d26e5c0417c933,205 Dundas St W,CA,Toronto,Canada,NaN,98,"[205 Dundas St W, Toronto ON M5G 1Z8, Canada]","[{'label': 'display', 'lat': 43.65484729461433...",43.654847,-79.387198,M5G 1Z8,ON,The Prince Shisha Cafe,v-1564848070


# Clean dataframe

In [45]:
df1 = dataframe
clean_columns = ['name', 'categories'] + [col for col in df1.columns if col.startswith('location.')]+ ['id']
clean_dataframe = df1.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Avenue Café + Bistro,Sandwich Place,480 University Ave,CA,Toronto,Canada,at Dundas St W,179,"[480 University Ave (at Dundas St W), Toronto ...","[{'label': 'display', 'lat': 43.65511700442771...",43.655117,-79.388764,M5G 1V2,ON,4b0d2ef1f964a520664423e3
1,The Fifth Cafe,Café,225 Richmond St. W,CA,Toronto,Canada,at Duncan St.,541,"[225 Richmond St. W (at Duncan St.), Toronto O...","[{'label': 'display', 'lat': 43.64934659382636...",43.649347,-79.389338,M5V 2C5,ON,52306d48498e7f7573f576fa
2,Cafe Plenty,Café,250 Dundas Street West,CA,Toronto,Canada,Simcoe Street,192,"[250 Dundas Street West (Simcoe Street), Toron...","[{'label': 'display', 'lat': 43.65457125894357...",43.654571,-79.389450,M5T 2Z5,ON,4f513029e4b07c3382c9fdb9
3,"Café De Paris, Downtown Toronto",Café,201 Dundas St W,CA,Toronto,Canada,NaN,102,"[201 Dundas St W, Toronto ON M5G 1C8, Canada]","[{'label': 'display', 'lat': 43.654872, 'lng':...",43.654872,-79.387429,M5G 1C8,ON,58e716e5fa7ed04f61194722
4,CafeAGO,Café,317 Dundas Street W,CA,Toronto,Canada,NaN,347,"[317 Dundas Street W, Toronto ON M5T 1G4, Canada]","[{'label': 'display', 'lat': 43.65405398907735...",43.654054,-79.391525,M5T 1G4,ON,4ad4c060f964a52064f720e3


In [22]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress', 'id'], axis=1)
clean_dataframe2

,name,categories,address,labeledLatLngs,lat,lng,postalCode,state
0,Avenue Café + Bistro,Sandwich Place,480 University Ave,"[{'label': 'display', 'lat': 43.65511700442771...",43.655117,-79.388764,M5G 1V2,ON
1,The Fifth Cafe,Café,225 Richmond St. W,"[{'label': 'display', 'lat': 43.64934659382636...",43.649347,-79.389338,M5V 2C5,ON
2,Cafe Plenty,Café,250 Dundas Street West,"[{'label': 'display', 'lat': 43.65457125894357...",43.654571,-79.389450,M5T 2Z5,ON
3,"Café De Paris, Downtown Toronto",Café,201 Dundas St W,"[{'label': 'display', 'lat': 43.654872, 'lng':...",43.654872,-79.387429,M5G 1C8,ON
4,CafeAGO,Café,317 Dundas Street W,"[{'label': 'display', 'lat': 43.65405398907735...",43.654054,-79.391525,M5T 1G4,ON
5,Queen Mother Cafe,Thai Restaurant,208 Queen St. W,"[{'label': 'display', 'lat': 43.650401, 'lng':...",43.650401,-79.389156,M5V 1Z2,ON
6,Cafe La Gaffe,Café,24 Baldwin St.,"[{'label': 'display', 'lat': 43.65621134762937...",43.656211,-79.393331,M5T 1L2,ON
7,One Pear Cafe,Fast Food Restaurant,201 Dundas Street West,"[{'label': 'display', 'lat': 43.65485, 'lng': ...",43.654850,-79.387330,M5G 1C8,ON
8,Krispy Kreme Doughnut Cafe,Donut Shop,162 McCaul St.,"[{'label': 'display', 'lat': 43.65577205054815...",43.655772,-79.391993,M5T 1W3,ON
9,The Prince Shisha Cafe,Smoke Shop,205 Dundas St W,"[{'label': 'display', 'lat': 43.65484729461433...",43.654847,-79.387198,M5G 1Z8,ON


In [46]:
# delete rows which its category is not a cafe
array= ['Café']
cafe_dataframe2 = clean_dataframe.loc[clean_dataframe2['categories'].isin(array)]
cafe_dataframe2

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
1,The Fifth Cafe,Café,225 Richmond St. W,CA,Toronto,Canada,at Duncan St.,541,"[225 Richmond St. W (at Duncan St.), Toronto O...","[{'label': 'display', 'lat': 43.64934659382636...",43.649347,-79.389338,M5V 2C5,ON,52306d48498e7f7573f576fa
2,Cafe Plenty,Café,250 Dundas Street West,CA,Toronto,Canada,Simcoe Street,192,"[250 Dundas Street West (Simcoe Street), Toron...","[{'label': 'display', 'lat': 43.65457125894357...",43.654571,-79.389450,M5T 2Z5,ON,4f513029e4b07c3382c9fdb9
3,"Café De Paris, Downtown Toronto",Café,201 Dundas St W,CA,Toronto,Canada,NaN,102,"[201 Dundas St W, Toronto ON M5G 1C8, Canada]","[{'label': 'display', 'lat': 43.654872, 'lng':...",43.654872,-79.387429,M5G 1C8,ON,58e716e5fa7ed04f61194722
4,CafeAGO,Café,317 Dundas Street W,CA,Toronto,Canada,NaN,347,"[317 Dundas Street W, Toronto ON M5T 1G4, Canada]","[{'label': 'display', 'lat': 43.65405398907735...",43.654054,-79.391525,M5T 1G4,ON,4ad4c060f964a52064f720e3
6,Cafe La Gaffe,Café,24 Baldwin St.,CA,Toronto,Canada,at Henry St,553,"[24 Baldwin St. (at Henry St), Toronto ON M5T ...","[{'label': 'display', 'lat': 43.65621134762937...",43.656211,-79.393331,M5T 1L2,ON,4aec87eff964a52058c821e3
10,Prince Cafe,Café,442 Dundas St W,CA,Toronto,Canada,NaN,106,"[442 Dundas St W, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.654916, 'lng':...",43.654916,-79.387351,NaN,ON,51bd60fa498e5ce3f52c6016
11,Cafe de Melbourne,Café,193 Church Street,CA,Toronto,Canada,Dundas,574,"[193 Church Street (Dundas), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65608780148982...",43.656088,-79.380705,NaN,ON,5655f6ca498ec7cc096eb46d
12,Cafe Bistro,Café,NaN,CA,NaN,Canada,NaN,262,[Canada],"[{'label': 'display', 'lat': 43.65605578874265...",43.656056,-79.385708,NaN,NaN,51b3697a498ecbc1c99a64f7
14,Light Café 路燈咖啡,Café,23 Baldwin Street,CA,Toronto,Canada,btwn Henry & McCaul St,523,"[23 Baldwin Street (btwn Henry & McCaul St), T...","[{'label': 'display', 'lat': 43.65607098189164...",43.656071,-79.393021,M5T 1L1,ON,5750c05e498ed2dde680c9b7
15,Uncle Tetsu's Japanese Angel Cafe,Café,191 Dundas St. West,CA,Toronto,Canada,At Centre Ave.,118,"[191 Dundas St. West (At Centre Ave.), Toronto...","[{'label': 'display', 'lat': 43.65500131144402...",43.655001,-79.386899,NaN,ON,5706b570498e618216dbfd32


In [51]:
from math import floor, ceil

latitude = cafe_dataframe2["lat"]
longitude = cafe_dataframe2["lng"]
data_points = [ np.array([i,j]) for i,j in zip(latitude, longitude)]


extension = 0.005
l_min = min(latitude) - extension
l_max = max(latitude) + extension
L_min = min(longitude) - extension
L_max = max(longitude) + extension

l_min, l_max, L_min, L_max

(43.644346593826356, 43.66247769310558, -79.39833144271435, -79.37570487601242)

# Generate Map

In [54]:
# Generate map to visualize cafe neighbourhood  
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

cafe_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(cafe_dataframe2['lat'], cafe_dataframe2['lng'], 
                                           cafe_dataframe2['name'], cafe_dataframe2['categories'],\
                                               cafe_dataframe2['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(cafe_map)  
    
cafe_map